<a href="https://colab.research.google.com/github/shahabbadihi/IT/blob/master/content-based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import pandas as pd
import requests
import io

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/PreprocessedDataset.csv"
response = requests.get(file_url)

dest = 'local-file.csv'

with open(dest, "wb") as filee:
  filee.write(response.content)

df = pd.read_csv(dest)

df.head(100)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages,cart_id
0,559850,خاک و کود تی تی بسته 10 کیلوگرمی,TC,466132,2,0,verified,not_recommended,خاک عاری از مواد مقوی,اصلا خاک خوبی نبود. کلی سنگ های درشت داشت و اص...,"[""ندارد""]","[""پر از سنگ ریزه ""]",1
1,112946,توستر فلر مدل TO 220,HA,466139,2,0,verified,not_recommended,خراب شدن اهرم پایین برنده,بعد از دو و نیم سال استفاده، اهرم پایین دهنده ...,"[""زیبایی، سبکی و اندازه مناسب توستر""]","[""خراب شدن اهرم پایین برنده""]",2
2,607671,هدست واقعیت مجازی ایکس پی پروداکت مدل VR BOX,AC,466160,1,2,verified,no_idea,قیمت پایین فدای کیفیت .....,کیفیت لنزش پایینه شرکت فروشنده گارانتی تعویض ن...,"[""ارزون""]","[""کیفیت پایین لنز""]",3
3,757892,کیف پول زنانه آیسا مدل KZP-1,FA,466160,1,0,verified,not_recommended,کیفیت ساخت پایین,اصلا با قیمت فروش ویژه هم ارزش خرید نداره_x000...,NaN,"[""کیفیت ساخت پایین\r"",""چسب کاری داخل و بیرون""]",3
4,601446,اسپیکر قابل حمل مدل ip085 مناسب برای آیفون و ...,IT,466160,2,2,verified,no_idea,بدرد ساختن اسپیکر پرتابل میخوره,دوستان من این مدل برای ساخت اسپیکر پرتابل گرفت...,"[""کیفیت قابل قبول برد و آمپیلی فایر داخلی\r"",""...","[""نداشتن بلوتوث""]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,600404,زیرپوش مردانه آستین کوتاه کد 002,FA,468143,1,0,verified,no_idea,زیرپوش مردانه آستین کوتاه,خنکه ولی بسیار نازکه.,"[""خنک بودن""]","[""بیش از حد نازک\r"",""بدون مارک""]",98
96,106164,چسب‌ پاک کننده بینی پیوردرم مدل Deep Cleansing...,PC,468143,2,9,verified,not_recommended,چسب‌ پاک کننده بینی پیوردرم,نسبت به خارجی‌های با کیفیت، خوب جوش‌های سر سیا...,"[""قیمت مناسب""]","[""عدم کارایی مناسب""]",96
97,251989,کیف دوربین ونگارد مدل Alta Rise 38,AV,468143,3,0,verified,\N,کیف ونگارد آلتا 38,بنده با توجه به اینکه یک عدد دوربین 5 دی و یک ...,"[""ضد آب بودن رویه\r"",""پدهای محافظ""]","[""کمتر از آنچه باید فضا دارد\r"",""کمی بزرگ و سن...",96
98,311053,ادو تویلت مردانه داویدوف مدل Cool Water Pacifi...,PC,468143,8,0,verified,no_idea,ادو تویلت مردانه داویدوف,من در پیشنهاد ویژه خریدم. علیرغم بوی خوبی که د...,NaN,NaN,97


In [ ]:
from pandas.core.frame import DataFrame
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import random
from scipy.sparse import csr_matrix

dff = DataFrame()
dff["user_id"] = df["user_id"]
dff["product_id"]= df["product_id"]
dff["content"] = df["comment"].str.cat(df["advantages"], sep = "-").str.cat(df["disadvantages"], sep = "-").str.cat(df["title"], sep = "-")
dff.head(100)

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/fa_stop_words.txt"
response = requests.get(file_url)

stop_words = []
with open("stopwords.txt", "wb") as filee:
  filee.write(response.content)

with open("stopwords.txt", "r") as filee:  
  for line in filee:
    stop_words.append(line)
stop_words = set(stop_words)

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(dff['content'].values.astype('U'))



# print(type(tfidf_matrix))

# for x in tfidf_matrix.columns:
#     tfidf_matrix.loc[(tfidf_matrix[x]<=0.01),x] = 0
# tfidf_matrix1 = csr_matrix(tfidf_matrix)
cos_sim = cosine_similarity(tfidf_matrix)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آباد', 'آخ', 'آخر', 'آخرها', 'آخه', 'آدمهاست', 'آرام', 'آره', 'آری', 'آزادانه', 'آسان', 'آسانی', 'آسیب', 'آشنایند', 'آشکارا', 'آقا', 'آقای', 'آقایان', 'آمد', 'آمدن', 'آمده', 'آمرانه', 'آن', 'آنان', 'آنانی', 'آنجا', 'آنرا', 'آنطور', 'آنقدر', 'آنكه', 'آنها', 'آنهاست', 'آنچنان', 'آنچه', 'آنکه', 'آنگاه', 'آهان', 'آهای', 'آور', 'آورد', 'آوردن', 'آورده', 'آوه', 'آيد', 'آی', 'آیا', 'آید', 'آیند', 'اتفاقا', 'اثر', 'اجراست', 'احتراما', 'احتمالا', 'احیانا', 'اخیر', 'اخیرا', 'اری', 'از', 'ازاین', 'ازجمله', 'ازش', 'اساسا', 'است', 'استفاد', 'استفاده', 'اسلامی', 'اش', 'اشتباها', 'اشکارا', 'اصلا', 'اصولا', 'اطلاعند', 'اعلام', 'اغلب', 'افزود', 'افسوس', 'اقل', 'اقلیت', 'اكنون', 'الا', 'الاسف', 'الان', 'البت', 'البته', 'الظاهر', 'الهی', 'الی', 'ام', 'اما', 'امروز', 'امروزه', 'امسال', 'امشب', 'امور', 'امیدوارم